In [1]:
import scanpy
import numpy as np
import matplotlib.pyplot as plt     
import pandas as pd
from tqdm import tqdm
import scipy as sp
import gc
import pickle

In [ ]:
# Retrieve the adata file from the figshare for tabula muris senis
adata = scanpy.read_h5ad("data/tabula-muris-senis-facs-processed-official-annotations.h5ad")
# homologs found with MGI (https://www.informatics.jax.org/batch) and the human TF database
homologs = pd.read_csv('data/homologs_csv.csv')
ages = adata.obs['age'].unique()
tissue_types = adata.obs['tissue'].unique()

/home/be278/.conda/envs/torch311/lib/python3.11/site-packages/anndata/_io/h5ad.py:267: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  return AnnData(**{
/home/be278/.conda/envs/torch311/lib/python3.11/site-packages/anndata/_io/h5ad.py:267: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  return AnnData(**{


In [ ]:
#homolgs = homologs[(homologs['Rank'])]# | (homologs['Rank']=='moderate') ]
TF_homolgs = homologs['Symbol'].unique() # Gets rid of blanks because not all TFs could be mapped
TF_set = set(TF_homolgs.tolist())
TF_homolgs.shape

['Zfp512b', 'Zfp821', 'Zfp652', 'Dmtf1l', 'Zrsr2-ps1', 'Zfp36l2', 'Gm7072', 'Cdkn2aip', 'Zfp532', 'Zfp395', 'Zfp764', 'Mybbp1a', 'Gm14391', 'Gc', 'Zfp950', 'AW146154', 'Zfp316', 'Sp140l2', 'Zfp524', 'Zfp296', 'Zfp575', 'Zfp397', 'Zfp951', 'Zfp282', 'Zfp9', 'Zfp768', 'Cdc5lrt8', 'Zfp558', 'Zfp229', 'Mterf1b', 'Zfp365', 'Zfp1002', 'Zfp207', 'Rhox13', 'Pdhx', 'Kcnh8', 'Asic2', 'Zscan4e', 'Zfp770', 'Zfp746', 'Zfp971', 'Zfp609', 'Rhox3e', 'Zfp800', 'Zfp865', 'Zfp335', 'Alg10', 'Zfp451', 'Zfp281', 'Rhox2e', 'Zfp346', 'Gm17067', 'Zfp13', 'Serpina3m', 'Zfp595', 'Zfp825', 'Zfp114', 'te', 'Zfp804a', 'Slc25a37', 'Zfp664', 'Zfp641', 'Dand5', 'Slc2a4rg-ps', 'Zfp354c', 'Zfp672', 'Gm14444', 'Zfp729b', 'Zfp423', 'Zfp974', 'Zfp317', 'Rhox2c', 'Rhox3a', 'Zfp112', 'Zfp329', 'Zfp607b', 'Zfp142', 'Zfp418', 'Zfp180', 'Mapre3', 'Pmp22', '1700020L13Rik', 'Sp140l1', 'Flii', 'Zfp354b', 'Zfp428', 'Zfp618', 'Zfp772', 'Zfp518a', 'Rgl2', 'Zfp248', 'Zfp683', 'Zfp607a', 'Serpina1c', 'Zfp106', 'Peds1', 'Duxf4', 'Zfp93

((1548,), (1639,), (1639, 37), 483, 574)

## Code to actually figure out which tissues and stuff to use

In [ ]:

# Important to note that this current code is taking a subset of the data with selected cell types
# You can change the dataset to include cell types of interest to you or get rid of the final
# line for getting the object "tiss" and it will include everything
df_selection = pd.read_csv("data/cell_types_subset_all.txt")
TF_set = set(TF_homolgs.tolist())

ntfs = np.zeros(len(tissue_types), dtype=int)
ncs3 = np.zeros(len(tissue_types), dtype=int)
ncs24 = np.zeros(len(tissue_types), dtype=int)

for i, t in enumerate(tissue_types):
    # filter the tissue and cell types
    selec_ct = df_selection.loc[df_selection["tissue"] == t, "cell_type"].unique()

    # .copy() avoids AnnData view chains holding memory IMPORTANT
    tiss = adata[adata.obs["tissue"] == t].copy()
    tiss = tiss[(tiss.obs["n_genes"] >= 250) & (tiss.obs["n_counts"] >= 5000)].copy()
    tiss = tiss[tiss.obs["cell_ontology_class"].isin(selec_ct)].copy()

    # seperate tf and tgs
    var_names = tiss.var_names
    tf_names   = var_names[var_names.isin(TF_set)] # tfs contained in data
    gene_names = var_names[~var_names.isin(TF_set)] # all the other genes in the set

    # seperate ages
    a3  = tiss[tiss.obs["age"] == "3m"].copy()
    a18 = tiss[tiss.obs["age"] == "18m"].copy()
    a24 = tiss[tiss.obs["age"] == "24m"].copy()

    # then split into tfs and tgs
    tf3  = a3[:, tf_names]
    tf18 = a18[:, tf_names]
    tf24 = a24[:, tf_names]

    tg3  = a3[:, gene_names]
    tg18 = a18[:, gene_names]
    tg24 = a24[:, gene_names]

    # Filter without importing huge arr
    # i.e. it works for sparse matrices.
    def expressed_mask(X, min_cells=3):
        if sp.sparse.issparse(X):
            # X.getnnz(axis=0) counts nonzeros per column
            return np.asarray(X.getnnz(axis=0) >= min_cells).ravel()
        else:
            return (X > 0).sum(axis=0) >= min_cells

    # get indices
    # Uncomment 18m stuff if you want to include this in analysis
    tf_mask = (
        expressed_mask(tf3.X, 3)
        #| expressed_mask(tf18.X, 3)
        | expressed_mask(tf24.X, 3)
    )

    tg_mask = (
        expressed_mask(tg3.X, 3)
        #| expressed_mask(tg18.X, 3)
        | expressed_mask(tg24.X, 3)
    )

    # filter
    tf3_f  = tf3[:, tf_mask]
    tf18_f = tf18[:, tf_mask]
    tf24_f = tf24[:, tf_mask]

    tg3_f  = tg3[:, tg_mask]
    tg18_f = tg18[:, tg_mask]
    tg24_f = tg24[:, tg_mask]

    ntfs[i] = tf3_f.n_vars
    ncs3[i] = tf3_f.n_obs
    ncs24[i] = tf24_f.n_obs

    print(
        t,
        "TF:", (tf3_f.n_obs, tf3_f.n_vars), (tf24_f.n_obs, tf24_f.n_vars),
        "TG:", (tg3_f.n_obs, tg3_f.n_vars), (tg24_f.n_obs, tg24_f.n_vars),
    )

    # ---- Hard cleanup per tissue
    del tiss, a3, a18, a24, tf3, tf18, tf24, tg3, tg18, tg24
    del tf3_f, tf18_f, tf24_f, tg3_f, tg18_f, tg24_f
    gc.collect()

Skin TF: (2336, 1155) (1104, 1155) TG: (2336, 15988) (1104, 15988)
Pancreas TF: (1448, 1170) (880, 1170) TG: (1448, 16804) (880, 16804)
Limb_Muscle TF: (799, 1070) (932, 1070) TG: (799, 14391) (932, 14391)
Heart TF: (3824, 1232) (2400, 1232) TG: (3824, 17821) (2400, 17821)
Spleen TF: (1343, 1026) (864, 1026) TG: (1343, 14514) (864, 14514)
Diaphragm TF: (441, 964) (360, 964) TG: (441, 13280) (360, 13280)
Trachea TF: (466, 1113) (175, 1113) TG: (466, 15166) (175, 15166)
Tongue TF: (1418, 1156) (1358, 1156) TG: (1418, 16245) (1358, 16245)
Thymus TF: (1346, 1122) (1205, 1122) TG: (1346, 15698) (1205, 15698)
Brain_Non-Myeloid TF: (2692, 1215) (509, 1215) TG: (2692, 16980) (509, 16980)
Brain_Myeloid TF: (4488, 1197) (4267, 1197) TG: (4488, 17260) (4267, 17260)
Bladder TF: (1306, 1134) (434, 1134) TG: (1306, 15728) (434, 15728)
Large_Intestine TF: (3987, 1368) (1955, 1368) TG: (3987, 20383) (1955, 20383)
BAT TF: (84, 918) (212, 918) TG: (84, 12105) (212, 12105)
GAT TF: (700, 1124) (457, 1124)

ages = ['3m', '24m']
#np.load('data/tissue_order_by_MI_prop.npy', allow_pickle=True)
df_selection = pd.read_csv('data/cell_types_subset_all.txt')
cell_types = adata.obs['cell_ontology_class'].unique()

dict_all_t={}
ntfs = np.zeros(len(tissue_types))
count=0
for t in tissue_types:
    # filter tissue specific
    dict_idxs = {}

    # Standard filter tissue by tissue
    selec_ct = df_selection[df_selection['tissue']==t]['cell_type'].unique()
    tiss_data = adata[adata.obs['tissue']==t]
    tiss_final = tiss_data[(tiss_data.obs["n_genes"] >= 250) & (tiss_data.obs["n_counts"] >= 5000)] 
    tiss_filt = tiss_final[tiss_final.obs['cell_ontology_class'].isin(selec_ct)]
    
    tf_names = tiss_filt.var['n_cells'].index[tiss_filt.var['n_cells'].index.isin(TF_homolgs.str.capitalize())]
    gene_names = tiss_filt.var['n_cells'].index[~tiss_filt.var['n_cells'].index.isin(TF_homolgs.str.capitalize())]
    
    adata3m=tiss_filt[tiss_filt.obs['age']=='3m']
    adata18m=tiss_filt[tiss_filt.obs['age']=='18m']
    adata24m=tiss_filt[tiss_filt.obs['age']=='24m']

    tf3m = adata3m[:, tf_names]; tf18m = adata18m[:, tf_names]; tf24m = adata24m[:, tf_names]
    tg3m = adata3m[:, gene_names]; tg18m = adata18m[:, gene_names]; tg24m = adata24m[:, gene_names]

    tf3m_X = tf3m.X.toarray(); tg3m_X = tg3m.X.toarray()
    tf3m_bin = tf3m_X; tg3m_bin = tg3m_X
    tf3m_bin[tf3m_bin>0] = 1; tg3m_bin[tg3m_bin>0] = 1

    tf18m_X = tf3m.X.toarray(); tg18m_X = tg18m.X.toarray()
    tf18m_bin = tf3m_X; tg18m_bin = tg18m_X
    tf18m_bin[tf18m_bin>0] = 1; tg18m_bin[tg18m_bin>0] = 1

    tf24m_X = tf24m.X.toarray(); tg24m_X = tg24m.X.toarray()
    tf24m_bin = tf24m_X; tg24m_bin = tg24m_X
    tf24m_bin[tf24m_bin>0] = 1; tg24m_bin[tg24m_bin>0] = 1

    tf_mask = (tf3m_bin.sum(axis=0) >= 3) | (tf18m_bin.sum(axis=0) >= 3) | (tf24m_bin.sum(axis=0) >= 3)
    tg_mask = (tg3m_bin.sum(axis=0) >= 3) | (tg18m_bin.sum(axis=0) >= 3) | (tg24m_bin.sum(axis=0) >= 3)

    tf3m_final = tf3m_bin[:,tf_mask]; tf18m_final = tf18m_bin[:,tf_mask]; tf24m_final = tf24m_bin[:,tf_mask] 
    tg3m_final = tg3m_bin[:,tg_mask]; tg18m_final = tg18m_bin[:,tg_mask]; tg24m_final = tg24m_bin[:,tg_mask] 

    ntfs[count] = tf3m_final.shape[1]
    count+=1
    print(t, tf3m_final.shape, tf24m_final.shape, tg3m_final.shape, tg24m_final.shape)


In [ ]:
# This is to just get a vibe on what the tissues look like 
ntfs, ncs3, ncs24, len(tissue_types[((ntfs>800) & (ncs3>700)) & (ncs24>700)])

(array([1155, 1170, 1070, 1232, 1026,  964, 1113, 1156, 1122, 1215, 1197,
        1134, 1368,  918, 1124, 1082, 1161, 1012, 1034, 1240, 1028, 1212,
        1153]),
 array([2336, 1448,  799, 3824, 1343,  441,  466, 1418, 1346, 2692, 4488,
        1306, 3987,   84,  700,  605,  793,  301,  637, 4170,  445,  280,
        1900]),
 array([1104,  880,  932, 2400,  864,  360,  175, 1358, 1205,  509, 4267,
         434, 1955,  212,  457,  364,  479,  263,  584, 3506,  530,  188,
           0]),
 10)

In [ ]:
# This is a mask enforcing some criteria. Here I say I need tissues with
# more than 700 TFs and more than 700 cells in each age
mask = (ntfs>700) & (ncs3>700)
mask2 = mask & (ncs24>700)
good_tissues = tissue_types[mask2]
np.save('data/good_tissues', good_tissues)
len(good_tissues)

10

In [69]:
print(list(good_tissues))
arr = np.load('data/tissue_order_by_MI_prop.npy', allow_pickle=True)
arr2 = np.load('data/good_tissues.npy', allow_pickle=True)

np.intersect1d(arr, arr2), np.intersect1d(arr2, good_tissues)

['Skin', 'Pancreas', 'Limb_Muscle', 'Heart', 'Spleen', 'Tongue', 'Thymus', 'Brain_Myeloid', 'Large_Intestine', 'Marrow']


(array(['Large_Intestine', 'Limb_Muscle', 'Pancreas', 'Skin', 'Thymus',
        'Tongue'], dtype=object),
 array(['Brain_Myeloid', 'Heart', 'Large_Intestine', 'Limb_Muscle',
        'Marrow', 'Pancreas', 'Skin', 'Spleen', 'Thymus', 'Tongue'],
       dtype=object))

## Code to write data given wanted tissues

In [ ]:

ages = ["3m", "18m", "24m"]

df_selection = pd.read_csv("data/cell_types_subset_all.txt")
TF_set = set(TF_homolgs.tolist())

ntfs = np.zeros(len(tissue_types), dtype=int)
ncs3 = np.zeros(len(tissue_types), dtype=int)
ncs24 = np.zeros(len(tissue_types), dtype=int)

tissues_filt = np.load('data/good_tissues.npy', allow_pickle=True)


for i, t in enumerate(tissues_filt):
    # filter the tissue and cell types
    selec_ct = df_selection.loc[df_selection["tissue"] == t, "cell_type"].unique()

    # .copy() avoids AnnData view chains holding memory IMPORTANT
    tiss = adata[adata.obs["tissue"] == t].copy()
    tiss = tiss[(tiss.obs["n_genes"] >= 250) & (tiss.obs["n_counts"] >= 5000)].copy()
    tiss = tiss[tiss.obs["cell_ontology_class"].isin(selec_ct)].copy()

    # seperate tf and tgs
    var_names = tiss.var_names
    tf_names   = var_names[var_names.isin(TF_set)] # tfs contained in data
    gene_names = var_names[~var_names.isin(TF_set)] # all the other genes in the set

    # seperate ages
    a3  = tiss[tiss.obs["age"] == "3m"].copy()
    a18 = tiss[tiss.obs["age"] == "18m"].copy()
    a24 = tiss[tiss.obs["age"] == "24m"].copy()

    # then split into tfs and tgs
    tf3  = a3[:, tf_names]
    tf18 = a18[:, tf_names]
    tf24 = a24[:, tf_names]

    tg3  = a3[:, gene_names]
    tg18 = a18[:, gene_names]
    tg24 = a24[:, gene_names]

    # Function filters for tfs expressed in >=3 cells without having to import massive arr
    # i.e. it works for sparse matrices.
    def expressed_mask(X, min_cells=3):
        if sp.sparse.issparse(X):
            # X.getnnz(axis=0) counts nonzeros per column
            return np.asarray(X.getnnz(axis=0) >= min_cells).ravel()
        else:
            return (X > 0).sum(axis=0) >= min_cells

    # =========================
    # This is basically saying keep a TF if it is present in 3 cells in at least 1 age
    # You can use 18m by uncommenting lines if you want but I will not here
    # ==========================
    tf_mask = (
        expressed_mask(tf3.X, 3)
        #| expressed_mask(tf18.X, 3)
        | expressed_mask(tf24.X, 3)
    )

    tg_mask = (
        expressed_mask(tg3.X, 3)
        #| expressed_mask(tg18.X, 3)
        | expressed_mask(tg24.X, 3)
    )

    # Filter the data 
    tf3_f  = tf3[:, tf_mask]
    tf18_f = tf18[:, tf_mask]
    tf24_f = tf24[:, tf_mask]

    tg3_f  = tg3[:, tg_mask]
    tg18_f = tg18[:, tg_mask]
    tg24_f = tg24[:, tg_mask]

    ntfs[i] = tf3_f.n_vars
    ncs3[i] = tf3_f.n_obs
    ncs24[i] = tf24_f.n_obs

    # In you uncomment below to get actual matrix...do this (to save space):
    def to_dense_binary(X):
        if sp.sparse.issparse(X):
            X = X.toarray()
        # This just stores it as a binary var so it takes up less space
        # Often this means you need to change data type while training model so watch out
        X = (X > 0).astype(np.uint8, copy=False)
        return X

    # Uncomment and use above function if you are saving data:
    tf3_bin  = to_dense_binary(tf3_f.X)
    tf24_bin = to_dense_binary(tf24_f.X)
    tg3_bin  = to_dense_binary(tg3_f.X)
    tg24_bin = to_dense_binary(tg24_f.X)
    # tf18_bin = to_dense_binary(tf18_f.X)
    # tg18_bin = to_dense_binary(tg18_f.X)

    np.save(f'data/data_bin_filt_{t}_3m_tf', tf3_bin)
    np.save(f'data/data_bin_filt_{t}_24m_tf', tf24_bin)
    np.save(f'data/data_bin_filt_{t}_3m_tg', tg3_bin)
    np.save(f'data/data_bin_filt_{t}_24m_tg', tg24_bin)
    #np.save(f'data/data_bin_filt_{t}_18m_tf', tf18_bin)
    #np.save(f'data/data_bin_filt_{t}_18m_tg', tg18_bin)

    print(
        t,
        "TF:", (tf3_f.n_obs, tf3_f.n_vars), (tf24_f.n_obs, tf24_f.n_vars),
        "TG:", (tg3_f.n_obs, tg3_f.n_vars), (tg24_f.n_obs, tg24_f.n_vars),
    )

    # Hard cleanup per tissue 
    # again NOTE that this stuff takes up an obnoxious amount of RAM so just try and make sure
    # you actually do this, especially on a remote server
    del tiss, a3, a18, a24, tf3, tf18, tf24, tg3, tg18, tg24
    del tf3_f, tf18_f, tf24_f, tg3_f, tg18_f, tg24_f
    gc.collect()

Skin TF: (2336, 1155) (1104, 1155) TG: (2336, 15988) (1104, 15988)
Pancreas TF: (1448, 1170) (880, 1170) TG: (1448, 16804) (880, 16804)
Limb_Muscle TF: (799, 1070) (932, 1070) TG: (799, 14391) (932, 14391)
Heart TF: (3824, 1232) (2400, 1232) TG: (3824, 17821) (2400, 17821)
Spleen TF: (1343, 1026) (864, 1026) TG: (1343, 14514) (864, 14514)
Tongue TF: (1418, 1156) (1358, 1156) TG: (1418, 16245) (1358, 16245)
Thymus TF: (1346, 1122) (1205, 1122) TG: (1346, 15698) (1205, 15698)
Brain_Myeloid TF: (4488, 1197) (4267, 1197) TG: (4488, 17260) (4267, 17260)
Large_Intestine TF: (3987, 1368) (1955, 1368) TG: (3987, 20383) (1955, 20383)
Marrow TF: (4170, 1240) (3506, 1240) TG: (4170, 18021) (3506, 18021)
